In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [5]:
# Replace these values with your actual MySQL login details
username = 'your_username'
password = 'your_password'
host = 'localhost'  # or IP address
port = '3306'
database = 'sakila'

# Create SQLAlchemy engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

In [8]:
pip install pymysql


Note: you may need to restart the kernel to use updated packages.


In [10]:
# Function to get Rentals for a Given Month
def rentals_month(engine, month, year):
    query = f"""
    SELECT customer_id, rental_date
    FROM rental
    WHERE MONTH(rental_date) = {month}
      AND YEAR(rental_date) = {year};
    """
    return pd.read_sql(query, engine)


In [12]:
# Count Rental Per Customer in a Month
def rental_count_month(df, month, year):
    count_df = df.groupby('customer_id').size().reset_index(name=f"rentals_{month:02}_{year}")
    return count_df


In [14]:
def compare_rentals(df1, df2):
    # Merge on customer_id, use outer join to capture customers in either month
    merged = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)

    # Get the column names
    col1 = df1.columns[1]
    col2 = df2.columns[1]

    # Add difference column
    merged['difference'] = merged[col2] - merged[col1]

    return merged
